In [1]:
# data analysis 
import pandas as pd
import numpy as np
import random as rnd
from sklearn.impute import SimpleImputer
import scipy.sparse as sp
import datetime as dt
# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', None)

In [2]:
attend = pd.read_csv('I:\\Download\\Датасеты\\attend.csv')
groups = pd.read_csv('I:\\Download\\Датасеты\\groups.csv')

In [3]:
groups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27003 entries, 0 to 27002
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   уникальный номер                27003 non-null  int64 
 1   направление 1                   27003 non-null  object
 2   направление 2                   27003 non-null  object
 3   направление 3                   27003 non-null  object
 4   адрес площадки                  27003 non-null  object
 5   округ площадки                  23021 non-null  object
 6   район площадки                  23021 non-null  object
 7   расписание в активных периодах  6522 non-null   object
 8   расписание в закрытых периодах  21643 non-null  object
 9   расписание в плановом периоде   67 non-null     object
dtypes: int64(1), object(9)
memory usage: 2.1+ MB


In [4]:
groups = groups.rename(columns={"уникальный номер": "numgroup", "направление 1": "type1", "направление 2": "type2", "направление 3": "type3", 
                       "адрес площадки": "addres", "округ площадки": "okrug", "район площадки": "region", "расписание в активных периодах": "active_schedule",
                      "расписание в закрытых периодах": "end_schedule", "расписание в плановом периоде": "plan_schedule"})

In [5]:
groups.head()

,numgroup,type1,type2,type3,addres,okrug,region,active_schedule,end_schedule,plan_schedule
0,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN
1,801356857,Физическая активность,ОФП,ОФП,"город Москва, Подольская улица, дом 5",Юго-Восточный административный округ,муниципальный округ Марьино,NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 10:00-11:...",NaN
2,801351684,Физическая активность,ОФП,ОФП,"г. Москва, Базовская улица, дом 15, строение 1...","Северный административный округ, Северный адми...","муниципальный округ Западное Дегунино, муницип...",NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 19:00-20:...",NaN
3,801353683,Физическая активность,ОФП,ОФП,"город Москва, улица Обручева, дом 28А, город М...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный...",NaN,"c 09.01.2023 по 31.03.2023, Пн., Ср. 13:30-14:...",NaN
4,801352164,Физическая активность,ОФП,ОФП,"город Москва, Воронцовский парк, дом 3, город ...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный...",NaN,"c 10.01.2023 по 28.02.2023, Вт., Пт. 12:00-13:...",NaN


In [6]:
groups.isnull().sum()

numgroup               0
type1                  0
type2                  0
type3                  0
addres                 0
okrug               3982
region              3982
active_schedule    20481
end_schedule        5360
plan_schedule      26936
dtype: int64

In [7]:
attend.head()

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,направление 2,направление 3,онлайн/офлайн,дата занятия,время начала занятия,время окончания занятия
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
1,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
2,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
3,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
4,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00


In [8]:
#Переименовываю столбцы
attend = attend.rename(columns={"уникальный номер занятия": "numzan", "уникальный номер группы": "numgroup", "уникальный номер участника": "numclient",
                               "направление 2": "type2", "направление 3": "type3", "онлайн/офлайн": "online", "дата занятия": "date",
                               "время начала занятия": "time_begin", "время окончания занятия": "time_end"})
attend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5901274 entries, 0 to 5901273
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   numzan      int64 
 1   numgroup    int64 
 2   numclient   int64 
 3   type2       object
 4   type3       object
 5   online      object
 6   date        object
 7   time_begin  object
 8   time_end    object
dtypes: int64(3), object(6)
memory usage: 405.2+ MB


In [9]:
#Смотрим пропущенные значения - пропущенных нет
attend.isnull().sum()

numzan        0
numgroup      0
numclient     0
type2         0
type3         0
online        0
date          0
time_begin    0
time_end      0
dtype: int64

In [10]:
#Значение Да/Нет преобразовываю в True/False
d = {'Нет':False, 'Да':True}
attend['online'] = attend['online'].map(d)
attend.head()

,numzan,numgroup,numclient,type2,type3,online,date,time_begin,time_end
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-01,09:00:00,10:00:00
1,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-01,09:00:00,10:00:00
2,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-01,09:00:00,10:00:00
3,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-01,09:00:00,10:00:00
4,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-01,09:00:00,10:00:00


In [11]:
attend['date'] = attend['date'].astype("datetime64[ns]") #Дату преобразовал в формат даты

In [12]:
columns_to_show = ['type2', 'type3', 'online']
attend.pivot_table(columns_to_show, ['numgroup'], aggfunc='count').head()

,online,type2,type3
numgroup,,,
801346549,30,30,30
801346550,855,855,855
801346551,451,451,451
801346552,388,388,388
801346553,2239,2239,2239


In [13]:
attend.numgroup.nunique() #Количество уникальных групп

24535

In [14]:
attend.numzan.nunique() #Количество уникальных занятий

647132

In [15]:
attend.numclient.nunique() #Уникальное количество пользователей всего

50906

In [16]:
# ТОП пользователей по группам 
attend\
    .groupby(['numclient', 'type2']) \
    .agg({'numgroup':'count'}) \
    .sort_values(by=['numgroup'], ascending=False).head() 

,,numgroup
numclient,type2,
101387204,ОНЛАЙН Художественно-прикладное творчество,743
101426166,ОНЛАЙН Рисование,628
101391680,ОНЛАЙН Рисование,621
101354085,ОНЛАЙН Художественно-прикладное творчество,569
101358683,ОНЛАЙН Рисование,567


In [17]:
# Количество пользователей посетивших различные занятия
attend.groupby(['type2', 'type3', 'online']).agg({"numclient": 'count'})\
    .sort_values(by=['numclient'], ascending=False).reset_index().head()

,type2,type3,online,numclient
0,ОНЛАЙН Красота и стиль,ОНЛАЙН Мастер-класс по уходу за кожей в зрелом...,True,413095
1,ОНЛАЙН Гимнастика,ОНЛАЙН Гимнастика,True,209704
2,Скандинавская ходьба,Скандинавская ходьба,False,182593
3,ОНЛАЙН Гимнастика,ОНЛАЙН Суставная гимнастика,True,173138
4,ОНЛАЙН Психология и коммуникации,ОНЛАЙН Психологические тренинги,True,145011


In [18]:
# ID пользователя с минимальной активностью за год
attend.groupby(['numclient'])['date'].nunique().idxmin()

101415587

In [19]:
# ID пользователя с максимальной активностью за год
attend.groupby(['numclient'])['date'].nunique().idxmax()

101420479

In [20]:
#Пользователь с максимальной активностью за год
attend.query('numclient == 101420479').groupby('type2').agg({'type3':'count'})\
    .sort_values(by=['type3'], ascending=False)

,type3
type2,
ОНЛАЙН Красота и стиль,229
ОНЛАЙН Гимнастика,146
ОНЛАЙН Здорово жить,127
ОНЛАЙН Психология и коммуникации,81
ОНЛАЙН ОФП,80


In [21]:
#Пользователь с минимальной активностью за год
attend.query('numclient == 101415587').groupby('type2').agg({'type3':'count'})\
    .sort_values(by=['type3'], ascending=False)

,type3
type2,
Информационные технологии,5


In [22]:
#Общее количество онлайн занятий и не онлайн
attend.groupby('online').count()

,numzan,numgroup,numclient,type2,type3,date,time_begin,time_end
online,,,,,,,,
False,2084345,2084345,2084345,2084345,2084345,2084345,2084345,2084345
True,3816929,3816929,3816929,3816929,3816929,3816929,3816929,3816929


In [23]:
print('Занятий онлайн: {:.2f}%'.format(100*(2084345/3816929)))

Занятий онлайн: 54.61%


In [24]:
# Онлайн программы по популярности
attend.query('online == True').groupby('type2').agg({'type3':'count'})\
    .sort_values(by=['type3'], ascending=False).head()

,type3
type2,
ОНЛАЙН Гимнастика,819496
ОНЛАЙН Красота и стиль,461259
ОНЛАЙН Здорово жить,346469
"ОНЛАЙН История, искусство, краеведение",294575
ОНЛАЙН Рисование,228054


In [25]:
# Офлайн программы по популярности
attend.query('online == False').groupby('type2').agg({'type3':'count'})\
    .sort_values(by=['type3'], ascending=False).head()

,type3
type2,
Гимнастика,408361
Танцы,223793
"Фитнес, тренажеры",210518
Скандинавская ходьба,182593
Спортивные игры,153462


In [26]:
#Смотрим за какой отрезок времени наш датасет - 333 дня - 1 год
attend['date'].max() - attend['date'].min()

Timedelta('333 days 00:00:00')

In [27]:
# Возвращаюсь к таблице groups
groups.head()

,numgroup,type1,type2,type3,addres,okrug,region,active_schedule,end_schedule,plan_schedule
0,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN
1,801356857,Физическая активность,ОФП,ОФП,"город Москва, Подольская улица, дом 5",Юго-Восточный административный округ,муниципальный округ Марьино,NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 10:00-11:...",NaN
2,801351684,Физическая активность,ОФП,ОФП,"г. Москва, Базовская улица, дом 15, строение 1...","Северный административный округ, Северный адми...","муниципальный округ Западное Дегунино, муницип...",NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 19:00-20:...",NaN
3,801353683,Физическая активность,ОФП,ОФП,"город Москва, улица Обручева, дом 28А, город М...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный...",NaN,"c 09.01.2023 по 31.03.2023, Пн., Ср. 13:30-14:...",NaN
4,801352164,Физическая активность,ОФП,ОФП,"город Москва, Воронцовский парк, дом 3, город ...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный...",NaN,"c 10.01.2023 по 28.02.2023, Вт., Пт. 12:00-13:...",NaN


In [28]:
groups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27003 entries, 0 to 27002
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   numgroup         27003 non-null  int64 
 1   type1            27003 non-null  object
 2   type2            27003 non-null  object
 3   type3            27003 non-null  object
 4   addres           27003 non-null  object
 5   okrug            23021 non-null  object
 6   region           23021 non-null  object
 7   active_schedule  6522 non-null   object
 8   end_schedule     21643 non-null  object
 9   plan_schedule    67 non-null     object
dtypes: int64(1), object(9)
memory usage: 2.1+ MB


In [29]:
# Оставляю нужные столбцы
groups = groups.loc[:, ['numgroup', 'addres', 'okrug', 'region']]
groups.head()

,numgroup,addres,okrug,region
0,801357270,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики
1,801356857,"город Москва, Подольская улица, дом 5",Юго-Восточный административный округ,муниципальный округ Марьино
2,801351684,"г. Москва, Базовская улица, дом 15, строение 1...","Северный административный округ, Северный адми...","муниципальный округ Западное Дегунино, муницип..."
3,801353683,"город Москва, улица Обручева, дом 28А, город М...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный..."
4,801352164,"город Москва, Воронцовский парк, дом 3, город ...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный..."


In [30]:
groups.isnull().sum()

numgroup       0
addres         0
okrug       3982
region      3982
dtype: int64

In [31]:
# Сопоставил номера групп из groups c attend на предмет наличия в attend
groups['numgroup'].isin(attend['numgroup']).value_counts()

True     24535
False     2468
Name: numgroup, dtype: int64

In [34]:
# Сохранил в одельный датафрейм только те номера групп, которые есть в attend
plan = groups.loc[groups['numgroup'].isin(attend['numgroup'].unique())]

In [38]:
plan.shape

(24535, 4)

In [40]:
# Приступим к юзерам
users = pd.read_csv('I:/Download/Датасеты/users.csv')
users = users.rename(columns={"уникальный номер": "numclient", "дата создание личного дела": "datereg", 
                              "пол": "sex", "дата рождения": "datebirth", 
                       "адрес проживания": "address_live"})
users.head()

,numclient,datereg,sex,datebirth,address_live
0,101391104,2019-02-26 15:52:09.000,Женщина,1959-09-10,"город москва, константинова, дом 30"
1,101437383,2022-09-14 13:46:41.079,Женщина,1958-06-29,"г. москва, большая серпуховская улица, дом 40,..."
2,101351338,2018-03-19 18:34:58.000,Женщина,1956-08-28,"город москва, летчика грицевца, дом 10"
3,101354578,2018-03-22 13:19:42.000,Женщина,1939-05-19,"город москва, лобачевского, дом 76"
4,101411997,2019-12-17 12:29:25.481,Женщина,1951-03-02,"г. москва, улица вертолётчиков, дом 13"


In [41]:
# Сопоставим юзеров с юзерами в attend
users['numclient'].isin(attend['numclient']).value_counts()

True     50906
False     1428
Name: numclient, dtype: int64

In [43]:
# Оставлю только тех, кто есть в attend
users2 = users.loc[users['numclient'].isin(attend['numclient'].unique())]
users2.shape

(50906, 5)

In [45]:
# Проверим пустоту
users2.isnull().sum()

numclient       0
datereg         0
sex             0
datebirth       0
address_live    0
dtype: int64

In [46]:
# Начинаю собирать датасет для модели
users_attend = attend.merge(users2, on='numclient', how='outer')
users_attend.head()

,numzan,numgroup,numclient,type2,type3,online,date,time_begin,time_end,datereg,sex,datebirth,address_live
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-01,09:00:00,10:00:00,2018-03-20 14:54:07.000,Женщина,1959-01-18,"город москва, кутузовский проспект, дом 19"
1,401348283,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-03,09:00:00,10:00:00,2018-03-20 14:54:07.000,Женщина,1959-01-18,"город москва, кутузовский проспект, дом 19"
2,401351970,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-08,09:00:00,10:00:00,2018-03-20 14:54:07.000,Женщина,1959-01-18,"город москва, кутузовский проспект, дом 19"
3,401355029,801350331,101352023,ОНЛАЙН Красота и стиль,ОНЛАЙН Мастер-класс по уходу за кожей в зрелом...,True,2022-08-08,18:30:00,19:30:00,2018-03-20 14:54:07.000,Женщина,1959-01-18,"город москва, кутузовский проспект, дом 19"
4,401355095,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,True,2022-08-10,09:00:00,10:00:00,2018-03-20 14:54:07.000,Женщина,1959-01-18,"город москва, кутузовский проспект, дом 19"


In [47]:
users_attend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5901274 entries, 0 to 5901273
Data columns (total 13 columns):
 #   Column        Dtype         
---  ------        -----         
 0   numzan        int64         
 1   numgroup      int64         
 2   numclient     int64         
 3   type2         object        
 4   type3         object        
 5   online        bool          
 6   date          datetime64[ns]
 7   time_begin    object        
 8   time_end      object        
 9   datereg       object        
 10  sex           object        
 11  datebirth     object        
 12  address_live  object        
dtypes: bool(1), datetime64[ns](1), int64(3), object(8)
memory usage: 590.9+ MB


In [48]:
users_attend.isnull().sum()

numzan          0
numgroup        0
numclient       0
type2           0
type3           0
online          0
date            0
time_begin      0
time_end        0
datereg         0
sex             0
datebirth       0
address_live    0
dtype: int64